# Create and run Earthquake Estimator

In [1]:
!nvidia-smi

Sat Apr  6 17:41:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
import densenet
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import pickle
%matplotlib inline

In [64]:
def reload_tf():
    global utils
    for i in range(2):
        import utils
        reload(utils)
        del utils
    
    global lstm_estimator
    global earthquake_input_fn
    global densenet
    global hooks
    global lstm
    global utils
    global process_inputs
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        import hooks
        import densenet
        import lstm
        import utils
        import process_inputs
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        reload(hooks)
        reload(densenet)
        reload(lstm)
        reload(utils)
        reload(process_inputs)
        del lstm_estimator
        del earthquake_input_fn
        del hooks
        del densenet
        del lstm
        del utils
        del process_inputs
    import lstm_estimator
    import earthquake_input_fn
    import densenet
    import hooks
    import lstm
    import utils
reload_tf()

<>:13: SyntaxWarning: name 'utils' is assigned to before global declaration
<>:13: SyntaxWarning: name 'utils' is assigned to before global declaration
<>:13: SyntaxWarning: name 'utils' is assigned to before global declaration
<ipython-input-64-da1f1c277902>:13: SyntaxWarning: name 'utils' is assigned to before global declaration
  global utils


In [10]:
def create_densenet_from_params(batch_size, timesteps, time_pool, growth_rate, layer_sizes, 
                                layer_dilations, layer_kernel_sizes, dropout_rate, compression_theta,
                                dense_size, feature_columns, optimizer_name, learning_rate, weight_decay,
                                momentum, nesterov, lambda_l2_reg, grad_clip, dtype,
                                use_lstm, lstm_layers, lstm_units, lstm_dropout,
                                model_dir=None):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'time_pool': time_pool,
        'growth_rate': growth_rate,
        'layer_sizes': layer_sizes,
        'layer_dilations': layer_dilations,
        'layer_kernel_sizes': layer_kernel_sizes,
        'dropout_rate': dropout_rate,
        'compression_theta': compression_theta,
        'dense_size': dense_size,
        'feature_columns': feature_columns,
        'optimizer_name': optimizer_name,
        'learning_rate': learning_rate,
        'weight_decay': weight_decay,
        'momentum': momentum,
        'nesterov': nesterov,
        'lambda_l2_reg': lambda_l2_reg,
        'grad_clip': grad_clip,
        'dtype': dtype,
        'use_lstm': use_lstm,
        'lstm_layers': lstm_layers,
        'lstm_units': lstm_units,
        'lstm_dropout': lstm_dropout
    }
    
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/densenet-%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    
    params_file = osp.join(osp.split(model_dir)[0], osp.split(model_dir)[1] + '.params.pickle')
    if not osp.isfile(params_file):
        print('writing params to %s' % params_file)
        with open(params_file, 'wb') as f:
            pickle.dump(params, f)
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(500 / batch_size),
                                    save_checkpoints_secs=300,
                                    session_config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.75)))
    
    estim = tf.estimator.Estimator(model_fn=densenet.densenet_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [11]:
def train_and_evaluate_densenet(batch_size, timesteps, time_pool, growth_rate, layer_sizes, 
                                layer_dilations, layer_kernel_sizes, dropout_rate, compression_theta,
                                dense_size, feature_columns, optimizer_name, learning_rate, weight_decay, momentum, nesterov,
                                lambda_l2_reg, grad_clip,
                                noise,
                                earthquake_data_dir,
                                eval_count,
                                eval_every_n_secs,
                                epochs=1,
                                dtype='float64',
                                use_lstm=False, lstm_layers=0, lstm_units=0, lstm_dropout=0,
                                model_dir=None):
    
    estim = create_densenet_from_params(batch_size=batch_size, timesteps=timesteps, time_pool=time_pool, 
                                        growth_rate=growth_rate, layer_sizes=layer_sizes,
                                        layer_dilations=layer_dilations, layer_kernel_sizes=layer_kernel_sizes,
                                        dropout_rate=dropout_rate, compression_theta=compression_theta,
                                        dense_size=dense_size, feature_columns=feature_columns, 
                                        optimizer_name=optimizer_name, learning_rate=learning_rate,
                                        weight_decay=weight_decay, momentum=momentum, nesterov=nesterov,
                                        lambda_l2_reg=lambda_l2_reg, grad_clip=grad_clip, dtype=dtype,
                                        use_lstm=use_lstm, lstm_layers=lstm_layers, lstm_units=lstm_units,
                                        lstm_dropout=lstm_dropout,
                                        model_dir=model_dir)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             noise,
                                                                             traintest='train',
                                                                             epochs=epochs),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           noise,
                                                                           traintest='test'),
                                     steps=eval_count,
                                     start_delay_secs=eval_every_n_secs, throttle_secs=eval_every_n_secs)
    
    return estim, trainspec, evalspec

In [84]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords5'
BATCH_SIZE = 32
TIMESTEPS = 150000
TIME_POOL = 125
GROWTH_RATE        = 2
LAYER_SIZES        = [12,  8]
LAYER_DILATIONS    = [ 1,  1]
LAYER_KERNEL_SIZES = [ 5,  3]
DROPOUT_RATE = 0.2
COMPRESSION_THETA = 0.5
DENSE_SIZE = 256
OPTIMIZER_NAME = 'MomentumW'
LEARNING_RATE = 0.01
WEIGHT_DECAY = 1e-4
MOMENTUM = 0.9
NESTEROV = True
LAMBDA_L2_REG = 0.0000  # weight decay instead
GRAD_CLIP = 1
NOISE = 0.00
EVAL_NUM_BATCHES = 200
EVAL_EVERY_N_SECONDS = 1800
EPOCHS = 10
DTYPE = 'float32'

USE_LSTM = True
LSTM_LAYERS = 2
LSTM_UNITS = 512
LSTM_DROPOUT = 0.2
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.float32, shape=TIMESTEPS)]

estim, train_spec, eval_spec = train_and_evaluate_densenet(BATCH_SIZE, TIMESTEPS, TIME_POOL, GROWTH_RATE,
                                                           LAYER_SIZES,
                                                           LAYER_DILATIONS, LAYER_KERNEL_SIZES,
                                                           DROPOUT_RATE, COMPRESSION_THETA, DENSE_SIZE,
                                                           FEATURE_COLUMNS,
                                                           OPTIMIZER_NAME, LEARNING_RATE, WEIGHT_DECAY,
                                                           MOMENTUM,
                                                           NESTEROV, LAMBDA_L2_REG, GRAD_CLIP,
                                                           NOISE,
                                                           EARTHQUAKE_DATA_DIR,
                                                           EVAL_NUM_BATCHES,
                                                           EVAL_EVERY_N_SECONDS,
                                                           EPOCHS, DTYPE,
                                                           USE_LSTM, LSTM_LAYERS, LSTM_UNITS, LSTM_DROPOUT,
                                                           # MODEL_DIR
                                                          )

writing params to /workspace/persistent-data/models/densenet-2019-04-08-12-35-59.params.pickle
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_task_type': 'worker', '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
, '_save_checkpoints_secs': 300, '_eval_distribute': None, '_master': '', '_num_worker_replicas': 1, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/workspace/persistent-data/models/densenet-2019-04-08-12-35-59', '_experimental_distribute': None, '_is_chief': True, '_protocol': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_device_fn': None, '_log_step_count_steps': 15, '_train_distribute': None, '_service': None, '_global_id_in_cluster': 0, '_evaluation_master': '', '_task_id': 0, '_num_ps_replicas': 0, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f716433cd68>}


In [85]:
try:
    tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)
except KeyboardInterrupt as e:
    raise e
except Exception as e:
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('--  TRAINING ERROR  --')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    print('----------------------')
    raise e

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
{'batch_size': 32, 'use_lstm': True, 'lstm_dropout': 0.2, 'layer_kernel_sizes': [5, 3], 'lstm_units': 512, 'timesteps': 150000, 'feature_columns': [_NumericColumn(key='acousticdata', shape=(150000,), default_value=None, dtype=tf.float32, normalizer_fn=None)], 'compression_theta': 0.5, 'layer_sizes': [12, 8], 'layer_dilations': [1, 1], 'learning_rate': 0.01, 'growth_rate': 2, 'momentum': 0.9, 'nesterov': True, 'weight_decay': 0.0001, 'lambda_l2_reg': 0.0, 'dense_size': 256, 'time_pool': 125, 'grad_clip': 1, 'dropout_rate': 0.2, 'lstm_layers': 2, 'dtype': 'float32', 'optimizer_name': 'MomentumW'}
stride_input_o1 Tens